https://golang.org/doc/codewalk/markov/

https://en.wikipedia.org/wiki/Markov_chain

https://en.wikipedia.org/wiki/N-gram#n-gram_models

In [124]:
import nltk
import random

# create a dict with the ngram model, it receives a list with the samples as string
ngram_dict = {}

def create_trigram_dict(corpus):
    n = 3
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] + " " + grams[:-1][1]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])

In [125]:
def generate_trigram(seed, samples = 15):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        seed = seed.split(" ")[1] + " " + new_sample

    return output

In [127]:
word_corpus = 'am I a gram or am I a markov chain am I a gram only ... maybe I am both'
ngram_dict = {}
create_trigram_dict(word_corpus.split(" "))
ngram_dict

{'... maybe': ['I'],
 'I a': ['gram', 'markov', 'gram'],
 'I am': ['both'],
 'a gram': ['or', 'only'],
 'a markov': ['chain'],
 'am I': ['a', 'a', 'a'],
 'chain am': ['I'],
 'gram only': ['...'],
 'gram or': ['am'],
 'markov chain': ['am'],
 'maybe I': ['am'],
 'only ...': ['maybe'],
 'or am': ['I']}

In [129]:
print generate_trigram("am I", 100)

am I a markov chain am I a gram or am I a gram or am I a gram only ... maybe I am both


In [3]:
import pickle

midi_dataset = pickle.load(open("validation_DB12_final_cleaned.p", "rb"))
print("ready!!!")

ready!!!


In [4]:
corpus = midi_dataset["x"]
corpus_str=map(str, corpus)

In [118]:
ngram_dict = {}
create_trigram_dict(corpus_str)
print("ready!!!")

ready!!!


In [119]:
generated_melody = generate_trigram("60 62", 40)
print(generated_melody)

60 62 70 69 74 73 71 73 72 73 72 70 67 68 68 68 70 70 70 71 70 68 68 67 65 67 67 67 69 64 81 83 84 86 88 83 86 84 86 87 89 91


In [20]:
from midiutil.MidiFile import MIDIFile   

# takes a list of integers representing midi notes and creates a .mid
# with a contant time of a quarter note for all the notes and 120 
# as tempo(taken from garcia, 2018)
def sequenceVector2midiMelody(seqVector, file_dir):
    MyMIDI = MIDIFile(1)
    track = 0 
    time = 0
    MyMIDI.addTrackName(track,time,"Sample Track") 
    MyMIDI.addTempo(track,time,120)
    time = 0
    for note in seqVector:
        # MyMIDI.addNote(track,channel,pitch,time,duration,volume)
        MyMIDI.addNote(0,0,note,time,1,100)
        time = time + 1

    binfile = open(file_dir, 'wb') 
    MyMIDI.writeFile(binfile) 
    binfile.close()

In [21]:
sequenceVector2midiMelody(map(int,generated_melody.split(" ")), 'generated_melody.mid')

For the next part is necessary to have the musescore software installed, you can do it with the comman sudo apt-get install musescore, and then replace the uri in the 2 lines of environment, it will usually be: "/usr/bin/musescore"

In [ ]:
from music21 import midi
import music21
music21.environment.set("musicxmlPath", "/usr/bin/musescore")
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

mid_file = midi.MidiFile()
mid_file.open("generated_melody.mid")
mid_file.read()
mid_file.close()
mid_stream = midi.translate.midiFileToStream(mid_file)
mid_stream.show()

mid_stream.show("midi")
sp = midi.realtime.StreamPlayer(mid_stream)
sp.play()

# generalizing to n-gram

In [104]:
import nltk
import random

# create a dict with the ngram model, it receives a list with the samples as string, an the n pramater
# of the n-gram
ngram_dict = {}

def create_ngram_dict(corpus, n = 3):
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] 
        for i in range(1, n-1):
             dict_key += " " + grams[:-1][i]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])
            
def generate_ngram(seed, samples = 15):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        temp_seed = seed.split(" ")[1]
        #import pdb
        #pdb.set_trace()
        for sample in seed.split(" ")[2:]:
            temp_seed += " " + sample 
        seed = temp_seed + " " + new_sample

    return output            

In [120]:
ngram_dict = {}
create_ngram_dict(corpus_str, 5)
print("ready!!!")

ready!!!


In [123]:
generated_melody = generate_ngram("60 62 60 62", 40)
print(generated_melody)

60 62 60 62 64 65 64 62 60 53 55 57 55 60 60 55 55 55 57 57 58 53 58 58 57 58 60 62 63 65 66 63 66 65 61 61 61 63 61 63 61 60 58 63


In [112]:
word_corpus = 'am I a gram or am I a markov chain ... maybe I am both'
ngram_dict = {}
create_ngram_dict(word_corpus.split(" "), 5)
ngram_dict

{'... maybe I am': ['both'],
 'I a gram or': ['am'],
 'I a markov chain': ['...'],
 'a gram or am': ['I'],
 'a markov chain ...': ['maybe'],
 'am I a gram': ['or'],
 'am I a markov': ['chain'],
 'chain ... maybe I': ['am'],
 'gram or am I': ['a'],
 'markov chain ... maybe': ['I'],
 'or am I a': ['markov']}

In [106]:
print generate_ngram("I a gram or", 100)

I a gram or am I a markov chain ... maybe I am both
